In [10]:
# Importación de librerías
import pandas as pd
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split, GridSearchCV

# Carga de Datos
print("Cargando datos...")
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")
print(f"Datos cargados: {len(train_data)} registros de entrenamiento, {len(test_data)} registros de prueba.")

# Formato de Surprise
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(train_data[['user', 'item', 'rating']], reader)

# División en conjunto de entrenamiento y validación
trainset, valset = train_test_split(data, test_size=0.2, random_state=42)

# Calcular medias de usuario y de ítem
user_means = train_data.groupby('user')['rating'].mean().to_dict()
item_means = train_data.groupby('item')['rating'].mean().to_dict()
global_mean = train_data['rating'].mean()

Cargando datos...
Datos cargados: 390351 registros de entrenamiento, 43320 registros de prueba.


In [12]:
# Definir el rango de hiperparámetros para GridSearch
param_grid = {
    'n_factors': [20, 50, 100],
    'n_epochs': [20, 50],
    'biased': [True],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.05]
}

print("Iniciando GridSearch...")
grid_search = GridSearchCV(SVD, param_grid, measures=['mae'], cv=3, n_jobs=-1)
grid_search.fit(data)

# Mejor combinación de hiperparámetros
best_params = grid_search.best_params['mae']
print(f"Mejor combinación de hiperparámetros: {best_params}")

# Crear el modelo SVD con los mejores hiperparámetros
best_svd = SVD(**best_params)
print("Entrenando el mejor modelo SVD...")
best_svd.fit(trainset)

Iniciando GridSearch...
Mejor combinación de hiperparámetros: {'n_factors': 20, 'n_epochs': 50, 'biased': True, 'lr_all': 0.002, 'reg_all': 0.02}
Entrenando el mejor modelo SVD...


In [17]:
import numpy as np
# Función para realizar predicciones con el mejor modelo SVD
def predict_rating(uid, iid):
    try:
        # Predicción usando el mejor modelo SVD
        pred = best_svd.predict(uid, iid).est
    except:
        pred = None

    # Si la predicción no es válida, utilizar medias
    if pred is None or np.isnan(pred) or pred == 0:
        if uid in user_means and iid in item_means:
            pred = (user_means[uid] + item_means[iid]) / 2
        elif uid in user_means:
            pred = user_means[uid]
        elif iid in item_means:
            pred = item_means[iid]
        else:
            pred = global_mean

    # Clamping entre 1 y 10
    # Clamping entre 1 y 10 con redondeo al entero más cercano y formato decimal ".0"
    return f"{round(max(1, min(10, pred)))}.0"



In [18]:
print("Generando predicciones para el conjunto de prueba...")
predictions = []

for _, row in test_data.iterrows():
    uid, iid, row_id = row['user'], row['item'], row['ID']
    pred_rating = predict_rating(uid, iid)
    predictions.append((row_id, pred_rating))

# Guardar las predicciones en CSV
print("Guardando archivo de predicciones...")
predictions_df = pd.DataFrame(predictions, columns=["ID", "rating"])
predictions_df.to_csv("predictions_svd_gridsearch.csv", index=False)
print("Archivo 'predictions_svd_gridsearch.csv' generado correctamente.")

Generando predicciones para el conjunto de prueba...
Guardando archivo de predicciones...
Archivo 'predictions_svd_gridsearch.csv' generado correctamente.
